In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec
import ta  # pip install ta

# Función para obtener datos de Binance
def get_data(symbol, interval='1h'):
    url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}'
    response = requests.get(url)
    data = pd.DataFrame(response.json(),
                        columns=['Open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time',
                                 'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume',
                                 'Taker_buy_quote_asset_volume', 'Ignore'])
    data['Date'] = pd.to_datetime(data['Open_time'], unit='ms')
    data.set_index('Date', inplace=True)
    return data.astype({'Open': float, 'High': float, 'Low': float, 'Close': float, 'Volume': float})

# Función para calcular indicadores técnicos
def calculate_indicators(data):
    data['SMA50'] = data['Close'].rolling(window=50).mean()
    data['SMA100'] = data['Close'].rolling(window=100).mean()
    data['EMA12'] = data['Close'].ewm(span=12).mean()
    data['EMA26'] = data['Close'].ewm(span=26).mean()
    data['Upper_BB'] = data['Close'].rolling(window=20).mean() + 2 * data['Close'].rolling(window=20).std()
    data['Lower_BB'] = data['Close'].rolling(window=20).mean() - 2 * data['Close'].rolling(window=20).std()
    data['MACD'] = data['EMA12'] - data['EMA26']
    data['Signal'] = data['MACD'].ewm(span=9).mean()
    delta = data['Close'].diff().dropna()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    data['High_Rolling'] = data['High'].rolling(50).max()
    data['Low_Rolling'] = data['Low'].rolling(50).min()
    data['Close_Shift'] = data['Close'].shift(1)
    data['Pivot'] = (data['High_Rolling'] + data['Low_Rolling'] + data['Close_Shift']) / 3
    data['R1'] = 2 * data['Pivot'] - data['Low_Rolling']
    data['S1'] = 2 * data['Pivot'] - data['High_Rolling']
    data['R2'] = data['Pivot'] + (data['High_Rolling'] - data['Low_Rolling'])
    data['S2'] = data['Pivot'] - (data['High_Rolling'] - data['Low_Rolling'])
    data['MFI'] = ta.volume.money_flow_index(data['High'], data['Low'], data['Close'], data['Volume'], window=14)
    return data

# Función auxiliar para determinar si un nivel está "cerca" del precio (umbral del 1%)
def prox(price, level, threshold=0.01):
    try:
        return abs(price - level) / level < threshold
    except:
        return False

# Función para crear la tabla de indicadores y recomendaciones (3 columnas) para cada temporalidad
def create_indicator_table(data):
    price = data['Close'].iloc[-1]
    sma_rec = "Alcista" if data['SMA50'].iloc[-1] > data['SMA100'].iloc[-1] else "Bajista"
    macd_rec = "Alcista" if data['MACD'].iloc[-1] > data['Signal'].iloc[-1] else "Bajista"
    senal_rec = "Confirma tendencia" if data['MACD'].iloc[-1] > data['Signal'].iloc[-1] else "No confirma"
    rsi_val = data['RSI'].iloc[-1]
    if rsi_val < 30:
        rsi_rec = "Compra (Sobrevendido)"
    elif rsi_val > 70:
        rsi_rec = "Venta (Sobrecomprado)"
    else:
        rsi_rec = "Neutral"
    mfi_val = data['MFI'].iloc[-1]
    if mfi_val < 20:
        mfi_rec = "Compra (Sobrevendido)"
    elif mfi_val > 80:
        mfi_rec = "Venta (Sobrecomprado)"
    else:
        mfi_rec = "Neutral"
    s1_val = data['S1'].iloc[-1]
    s2_val = data['S2'].iloc[-1]
    r1_val = data['R1'].iloc[-1]
    r2_val = data['R2'].iloc[-1]
    s1_rec = "Rebote potencial" if prox(price, s1_val) else "No cercano"
    s2_rec = "Rebote potencial" if prox(price, s2_val) else "No cercano"
    r1_rec = "Rechazo potencial" if prox(price, r1_val) else "No cercano"
    r2_rec = "Rechazo potencial" if prox(price, r2_val) else "No cercano"
    
    table_data = [
        ["SMA50", f"{data['SMA50'].iloc[-1]:.2f}", sma_rec],
        ["SMA100", f"{data['SMA100'].iloc[-1]:.2f}", sma_rec],
        ["MACD", f"{data['MACD'].iloc[-1]:.2f}", macd_rec],
        ["Señal MACD", f"{data['Signal'].iloc[-1]:.2f}", senal_rec],
        ["RSI (14)", f"{rsi_val:.2f}", rsi_rec],
        ["MFI (14)", f"{mfi_val:.2f}", mfi_rec],
        ["Soporte S1", f"{s1_val:.2f}", s1_rec],
        ["Soporte S2", f"{s2_val:.2f}", s2_rec],
        ["Resistencia R1", f"{r1_val:.2f}", r1_rec],
        ["Resistencia R2", f"{r2_val:.2f}", r2_rec]
    ]
    return table_data

# Función para dibujar soportes y resistencias en un gráfico, solo si están dentro de los límites del eje Y
def plot_supports_resistances(ax, data):
    ymin, ymax = ax.get_ylim()
    xlim = ax.get_xlim()
    for level_name, color in zip(["S1", "S2", "R1", "R2"], ["green", "green", "red", "red"]):
        level_value = data[level_name].iloc[-1]
        if ymin <= level_value <= ymax:
            ax.axhline(level_value, color=color, linestyle="--", linewidth=1)
            ax.text(xlim[1], level_value, f"{level_value:.2f}", 
                    verticalalignment="center", horizontalalignment="right", fontsize=8)

# Función para crear el resumen final de recomendaciones generales
def create_recommendations(data):
    sma50_above_sma100 = data['SMA50'].iloc[-1] > data['SMA100'].iloc[-1]
    sma50_cross_sma100 = data['SMA50'].iloc[-2] < data['SMA100'].iloc[-2] and sma50_above_sma100
    sma_direction = 'ALCISTA' if sma50_above_sma100 else 'BAJISTA'
    cross_direction = 'por encima de' if sma50_above_sma100 else 'por debajo de'
    macd_above_signal = data['MACD'].iloc[-1] > data['Signal'].iloc[-1]
    macd_cross_signal = data['MACD'].iloc[-2] < data['Signal'].iloc[-2] and macd_above_signal
    signal_direction = 'por encima de' if macd_above_signal else 'por debajo de'
    rsi = data['RSI'].iloc[-1]
    mfi = data['MFI'].iloc[-1]
    window = 20
    data['Middle_Band'] = data['Close'].rolling(window=window).mean()
    data['Upper_Band'] = data['Middle_Band'] + 2 * data['Close'].rolling(window=window).std()
    data['Lower_Band'] = data['Middle_Band'] - 2 * data['Close'].rolling(window=window).std()
    price = data['Close'].iloc[-1]
    upper_band = data['Upper_Band'].iloc[-1]

    # En algunos casos es posible que Lower_BB ya esté calculado en calculate_indicators, pero aquí lo usamos para mantener la consistencia
    lower_band = data['Lower_BB'].iloc[-1]
    middle_band = data['Middle_Band'].iloc[-1]
    decimal_format = '.6f' if price < 0.01 else (".4f" if price < 1 else '.2f')
    if sma50_cross_sma100:
        recommendation = 'BUY'
        confidence_level = 0.8
    elif macd_cross_signal:
        recommendation = 'BUY'
        confidence_level = 0.7
    elif rsi < 30:
        recommendation = 'BUY'
        confidence_level = 0.6
    elif mfi < 20:
        recommendation = 'BUY'
        confidence_level = 0.5
    elif price > upper_band:
        recommendation = 'SELL'
        confidence_level = 0.8
    elif price < lower_band:
        recommendation = 'BUY'
        confidence_level = 0.8
    else:
        recommendation = 'HOLD'
        confidence_level = 0.5
    recommendation_text = (
        f"1. Soportes: S1: {data['S1'].iloc[-1]:{decimal_format}}, S2: {data['S2'].iloc[-1]:{decimal_format}}\n"
        f"2. Resistencias: R1: {data['R1'].iloc[-1]:{decimal_format}}, R2: {data['R2'].iloc[-1]:{decimal_format}}\n"
        f"3. La SMA50 está {cross_direction} la SMA100, indicando tendencia {sma_direction}.\n"
        f"4. La línea MACD está {signal_direction} la línea de señal.\n"
        f"5. RSI: {rsi:.2f}\n"
        f"6. MFI: {mfi:.2f}\n"
        f"7. Precio: {price:{decimal_format}}, BB-UP: {upper_band:{decimal_format}}, "
        f"BB-MID: {middle_band:{decimal_format}}, BB-DOWN: {lower_band:{decimal_format}}\n"
        f"\nRecomendación General: {recommendation}\nNivel de confianza: {round(confidence_level * 100)}%"
    )
    return recommendation_text

# --- Configuración y generación del PDF ---

criptomonedas = ["BTCUSDT", "ETHUSDT"]
pdf = PdfPages("analisis_criptomonedas.pdf")

for simbolo in criptomonedas:
    # Obtener datos para cada temporalidad
    datos_horarios = get_data(simbolo, interval="1h")
    datos_diarios   = get_data(simbolo, interval="1d")
    datos_semanales = get_data(simbolo, interval="1w")
    
    if datos_horarios.empty or datos_diarios.empty or datos_semanales.empty:
        print(f"No hay datos para {simbolo}, se omite.")
        continue
    
    # Calcular indicadores para cada conjunto
    datos_horarios = calculate_indicators(datos_horarios)
    datos_diarios   = calculate_indicators(datos_diarios)
    datos_semanales = calculate_indicators(datos_semanales)
    
    # Crear las tablas de indicadores para cada temporalidad
    table_data_hour = create_indicator_table(datos_horarios)
    table_data_daily = create_indicator_table(datos_diarios)
    table_data_weekly = create_indicator_table(datos_semanales)
    
    # Obtener resumen final (usamos datos diarios para el resumen)
    recommendation_text = create_recommendations(datos_diarios)
    
    # Crear la figura en formato landscape
    fig = plt.figure(figsize=(11.69, 8.27))
    # Título de la página con el nombre del par crypto
    fig.suptitle(f"Análisis de {simbolo}", fontsize=16, y=0.98)
    
    # Dividir la figura en 3 filas: gráficos, tablas y resumen final
    gs = gridspec.GridSpec(3, 1, height_ratios=[2, 1, 0.5])
    
    # Gráficos (1 fila, 3 columnas)
    chart_grid = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs[0], wspace=0.3)
    ax_hour = fig.add_subplot(chart_grid[0])
    ax_daily = fig.add_subplot(chart_grid[1])
    ax_weekly = fig.add_subplot(chart_grid[2])
    
    # Gráfico Horario
    ax_hour.plot(datos_horarios.index, datos_horarios["Close"], color='orange', label="Horario")
    ax_hour.set_title("Horario", fontsize=10)
    ax_hour.set_ylabel("Precio", fontsize=8)
    ax_hour.legend(loc="upper left", fontsize=8)
    plot_supports_resistances(ax_hour, datos_horarios)
    
    # Gráfico Diario
    ax_daily.plot(datos_diarios.index, datos_diarios["Close"], color='blue', label="Diario")
    ax_daily.set_title("Diario", fontsize=10)
    ax_daily.set_ylabel("Precio", fontsize=8)
    ax_daily.legend(loc="upper left", fontsize=8)
    plot_supports_resistances(ax_daily, datos_diarios)
    
    # Gráfico Semanal
    ax_weekly.plot(datos_semanales.index, datos_semanales["Close"], color='green', label="Semanal")
    ax_weekly.set_title("Semanal", fontsize=10)
    ax_weekly.set_ylabel("Precio", fontsize=8)
    ax_weekly.legend(loc="upper left", fontsize=8)
    ax_weekly.set_xlabel("Fecha", fontsize=8)
    plt.setp(ax_weekly.get_xticklabels(), rotation=45, fontsize=7)
    plot_supports_resistances(ax_weekly, datos_semanales)
    
    # Tablas (1 fila, 3 columnas)
    table_grid = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=gs[1], wspace=0.3)
    ax_table_hour = fig.add_subplot(table_grid[0])
    ax_table_daily = fig.add_subplot(table_grid[1])
    ax_table_weekly = fig.add_subplot(table_grid[2])
    
    for ax_tab, table_data, title in zip(
            [ax_table_hour, ax_table_daily, ax_table_weekly],
            [table_data_hour, table_data_daily, table_data_weekly],
            ["Indicadores Horarios", "Indicadores Diarios", "Indicadores Semanales"]):
        ax_tab.axis("off")
        tabla = ax_tab.table(cellText=table_data, 
                             colLabels=["Indicador", "Valor", "Recomendación"], 
                             loc="center")
        tabla.auto_set_font_size(False)
        tabla.set_fontsize(7)
        tabla.scale(1, 0.8)
        ax_tab.set_title(title, fontsize=9)
    
    # Resumen final debajo de las tablas
    ax_summary = fig.add_subplot(gs[2])
    ax_summary.axis("off")
    ax_summary.text(0.5, 0.5, recommendation_text, ha="center", va="center", fontsize=8, wrap=True)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    pdf.savefig(fig)
    plt.close(fig)

pdf.close()
print("PDF generado: analisis_criptomonedas.pdf")


PDF generado: analisis_criptomonedas.pdf
